### Cleaning Overview
- Input: Raw GA4 item-level Parquet snapshot
- Objective: Prepare analytics-ready dataset
- Key cleaning steps:
  - Null handling
  - Deduplication
  - Type standardization
  - Feature derivation
- Output: Clean Parquet for downstream analysis


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
use_sample = False

if use_sample == True:
    RAW_DATA_PATH = Path('../data/sample')
    raw_file = RAW_DATA_PATH / 'fact_products_raw_sample.parquet'
else:
    RAW_DATA_PATH = Path('../data/raw')
    raw_file = RAW_DATA_PATH / 'fact_products_raw.parquet'
    
CLEAN_DATA_PATH = Path('../data/cleaned')
df = pd.read_parquet(raw_file)
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3839535 entries, 0 to 3839534
Data columns (total 18 columns):
 #   Column           Non-Null Count    Dtype              
---  ------           --------------    -----              
 0   event_date       3839535 non-null  datetime64[ns]     
 1   event_ts         3839535 non-null  datetime64[us, UTC]
 2   event_name       3839535 non-null  object             
 3   user_id          3839535 non-null  object             
 4   transaction_id   14205 non-null    object             
 5   session_id       3839535 non-null  Int64              
 6   device           3839535 non-null  object             
 7   country          3839535 non-null  object             
 8   region           3839535 non-null  object             
 9   city             3839535 non-null  object             
 10  traffic_sname    3839535 non-null  object             
 11  traffic_smedium  3839535 non-null  object             
 12  traffic_source   3839535 non-null  object 

In [3]:
# deduplicating the dataframe

dedup_df = df.drop_duplicates(subset = ['user_id','item_id','event_name','event_ts','session_id']).copy()


In [4]:
# normalizing traffic columns

for col in ['traffic_smedium', 'traffic_source', 'traffic_sname']:
    dedup_df[col] = dedup_df[col].fillna('').str.lower().str.strip()


# derriving categories for marketing channel

conditions = [
    # privacy
    (dedup_df['traffic_smedium'] == '(data deleted)') |
    (dedup_df['traffic_source'] == '(data deleted)' ) |
    (dedup_df['traffic_sname'] == '(data deleted)'),
    
    # direct traffic
    (dedup_df['traffic_smedium'] == '(none)')         &
    (dedup_df['traffic_source'] == '(direct)'),

    # organic traffic
    (dedup_df['traffic_smedium'] == 'organic'),

    #paid
    (dedup_df['traffic_smedium'] == 'cpc'),

    #referral
    (dedup_df['traffic_smedium'] == 'referral')
]

choices = [
    'unknown_privacy',
    'direct',
    'organic',
    'paid',
    'referral'
]

# adding marketing channel category

dedup_df['marketing_channel'] = np.select(conditions,choices,default='others')


In [5]:
# normalize item names

item_name_norm = (dedup_df['item_name'].fillna('').str.lower())


product_type_conditions = [
    item_name_norm.str.contains(r'hoodie', regex = True),
    item_name_norm.str.contains(r'sweatshirt|crewneck|pullover', regex = True),
    item_name_norm.str.contains(r'jacket|vest|shell|rain', regex = True),
    item_name_norm.str.contains(r'\btee\b|t-shirt|tshirt', regex = True),
    item_name_norm.str.contains(r'onesie', regex = True),
    item_name_norm.str.contains(r'cap|hat|beanie', regex = True),
    item_name_norm.str.contains(r'sock', regex = True),
    item_name_norm.str.contains(r'bottle|tumbler|cup', regex = True),
    item_name_norm.str.contains(r'tote|pack|bag', regex = True),
    item_name_norm.str.contains(r'journal|book|note', regex = True),
    item_name_norm.str.contains(r'sticker|decal', regex = True),
    item_name_norm.str.contains(r'pin', regex = True),
    item_name_norm.str.contains(r'gift', regex = True),
]
product_type_choice = [
    'hoodie',
    'sweatshirt',
    'outerwear',
    't-shirt',
    'onesie',
    'headwear',
    'socks',
    'drinkware',
    'bags',
    'stationery',
    'stickers',
    'pin',
    'gift cards'
]

# adding product types

dedup_df['product_type'] = np.select(product_type_conditions,product_type_choice, default = "others")

In [6]:
target_segment_conditions = [
    item_name_norm.str.contains('women'),
    item_name_norm.str.contains('men'),
    item_name_norm.str.contains('unisex'),
    item_name_norm.str.contains('youth'),
    item_name_norm.str.contains(r'toddler|infant', regex = True)
]

target_segment_choice = [
    'women',
    'men',
    'unisex',
    'youth',
    'toddler/infant'
]

# adding target_segment

dedup_df['target_segment'] = np.select(target_segment_conditions,target_segment_choice, default = "others")

In [7]:
brand_conditions = [
    item_name_norm.str.contains(r'#iamremarkable', regex = True),
    item_name_norm.str.contains('android'),
    item_name_norm.str.contains('youtube'),
]

brand_choice = [
    '#iamremarkable',
    'android',
    'youtube'
]

# adding brand

dedup_df['brand'] = np.select(brand_conditions,brand_choice, default = 'google')

In [9]:
# categorizing the columns and adding flags

category_columns = [
    'event_name', 'device','country','marketing_channel','product_type','target_segment','brand'
]

for col in category_columns:
    dedup_df[col] = dedup_df[col].astype('category')

dedup_df['is_purchase'] = dedup_df['transaction_id'].notna()

In [10]:
# exporting clean parquet file to cleaned.

OUTPUT_PATH = CLEAN_DATA_PATH / 'fact_product_cleaned.parquet'

dedup_df.to_parquet(OUTPUT_PATH, index = False)